In [2]:
import pandas as pd
import numpy as np

slippage = pd.read_csv('slippage.csv')


,Unnamed: 0,1600000_slippage,200000_slippage,400000_slippage,800000_slippage,amount,dai_to_usdc,date,liquidity_curve_susd,liquidity_curvefi,liquidity_oasis,liquidity_uniswap_v2,liquidity_zero_x,price,time
147,147,0.000000,0,0.000000,0.000000,1600000,1,2020-08-04,49,40,8.0,NaN,3.0,1.012608,09:11:05.564296
148,148,0.001914,0,0.000258,0.000792,200000,0,2020-08-04,47,53,0.0,NaN,NaN,0.985026,09:11:10.710394
149,149,0.001656,0,0.000000,0.000534,400000,0,2020-08-04,51,49,0.0,NaN,NaN,0.984768,09:11:10.710394
150,150,0.001122,0,0.000000,0.000000,800000,0,2020-08-04,53,47,0.0,NaN,NaN,0.984234,09:11:10.710394
151,151,0.000000,0,0.000000,0.000000,1600000,0,2020-08-04,53,43,4.0,NaN,NaN,0.983113,09:11:10.710394


In [5]:
# Organize df
# drop index
slippage = slippage.drop('Unnamed: 0', axis = 1)
# Reorder cols
slippage = slippage[['date','time','dai_to_usdc','amount','price','200000_slippage','400000_slippage','800000_slippage','1600000_slippage','liquidity_curve_susd','liquidity_curvefi','liquidity_oasis','liquidity_uniswap_v2','liquidity_zero_x']]
# Fill NAs
slippage = slippage.fillna(0)

,date,time,dai_to_usdc,amount,price,200000_slippage,400000_slippage,800000_slippage,1600000_slippage,liquidity_curve_susd,liquidity_curvefi,liquidity_oasis,liquidity_uniswap_v2,liquidity_zero_x
0,2020-08-04,00:08:13.842727,1,200000,1.014572,0,0.000279,0.000767,0.001563,53,45,0.0,NaN,2.0
1,2020-08-04,00:08:13.842727,1,400000,1.014293,0,0.000000,0.000488,0.001285,53,46,0.0,NaN,1.0
2,2020-08-04,00:08:13.842727,1,800000,1.013805,0,0.000000,0.000000,0.000797,48,41,7.0,NaN,4.0
3,2020-08-04,00:08:13.842727,1,1600000,1.013008,0,0.000000,0.000000,0.000000,48,43,6.0,NaN,3.0
4,2020-08-04,00:08:18.986078,0,200000,0.984384,0,0.000268,0.000829,0.001973,55,45,0.0,NaN,0.0


In [31]:
avg_slippage_200_400 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 1)]['400000_slippage'].mean()
avg_slippage_200_800 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 1)]['800000_slippage'].mean()
avg_slippage_200_1600 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 1)]['1600000_slippage'].mean()
avg_slippage_400_800 = slippage[(slippage['amount'] == 400000) & (slippage['dai_to_usdc'] == 1)]['800000_slippage'].mean()
avg_slippage_400_1600 = slippage[(slippage['amount'] == 400000) & (slippage['dai_to_usdc'] == 1)]['1600000_slippage'].mean()
avg_slippage_800_1600 = slippage[(slippage['amount'] == 800000) & (slippage['dai_to_usdc'] == 1)]['1600000_slippage'].mean()

In [32]:
dai_to_usdc_slippage = pd.DataFrame({'400k': [avg_slippage_200_400, np.nan, np.nan],
                                     '800k': [avg_slippage_200_800, avg_slippage_400_800, np.nan],
                                     '1600k': [avg_slippage_200_1600, avg_slippage_400_1600, avg_slippage_800_1600]}, index=['200k','400k','800k'])

print('DAI to USDC Average Slippage')
display(dai_to_usdc_slippage)

DAI to USDC Average Slippage


,400k,800k,1600k
200k,0.000276,0.000721,0.001515
400k,NaN,0.000445,0.001239
800k,NaN,NaN,0.000794


In [33]:
avg_slippage_200_400 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 0)]['400000_slippage'].mean()
avg_slippage_200_800 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 0)]['800000_slippage'].mean()
avg_slippage_200_1600 = slippage[(slippage['amount'] == 200000) & (slippage['dai_to_usdc'] == 0)]['1600000_slippage'].mean()
avg_slippage_400_800 = slippage[(slippage['amount'] == 400000) & (slippage['dai_to_usdc'] == 0)]['800000_slippage'].mean()
avg_slippage_400_1600 = slippage[(slippage['amount'] == 400000) & (slippage['dai_to_usdc'] == 0)]['1600000_slippage'].mean()
avg_slippage_800_1600 = slippage[(slippage['amount'] == 800000) & (slippage['dai_to_usdc'] == 0)]['1600000_slippage'].mean()

usdc_to_dai_slippage = pd.DataFrame({'400k': [avg_slippage_200_400, np.nan, np.nan],
                                     '800k': [avg_slippage_200_800, avg_slippage_400_800, np.nan],
                                     '1600k': [avg_slippage_200_1600, avg_slippage_400_1600, avg_slippage_800_1600]}, index=['200k','400k','800k'])

print('USDC to DAI Average Slippage')
display(usdc_to_dai_slippage)

USDC to DAI Average Slippage


,400k,800k,1600k
200k,0.000289,0.000832,0.001943
400k,NaN,0.000542,0.001653
800k,NaN,NaN,0.001111


In [35]:
print('Slippage Difference Between Trading Direction')

dai_to_usdc_slippage - usdc_to_dai_slippage


Slippage Difference Between Trading Direction


,400k,800k,1600k
200k,-0.000013,-0.000111,-0.000428
400k,NaN,-0.000098,-0.000415
800k,NaN,NaN,-0.000317


In [66]:
def analyze_slippage():
    # Run process for both trade directions
    dfs = {}
    for x in [0, 1]:
        merged_results = []
        for amount in [200000, 400000, 800000]:
            result = []
            for amount2 in [400000, 800000, 1600000]:
                result.append(slippage[(slippage['amount'] == amount) & (slippage['dai_to_usdc'] == x)][str(amount2) + '_slippage'].min())
            merged_results.append(result)

        merged_results = pd.DataFrame(merged_results, columns = ['400k','800k','1600k'], index = ['200k','400k','800k'])
        if x == 0:
            dfs['USDC to DAI'] = merged_results
        else:
            dfs['DAI to USDC'] = merged_results
    return dfs

def display_analysis(dict, descriptive_string):
    print('DAI to USDC ' + descriptive_string)
    display(dict['DAI to USDC'])
    print('\n\n')
    print('USDC to DAI ' + descriptive_string)
    display(dict['USDC to DAI'])
    print('\n\n')
    print('Difference')
    display(dict['DAI to USDC'] - dict['USDC to DAI'])
analysis = analyze_slippage()
display_analysis(analysis, 'Minimum Slippage')

DAI to USDC Minimum Slippage


,400k,800k,1600k
200k,0.000197,0.000599,0.001354
400k,0.000000,0.000396,0.001155
800k,0.000000,0.000000,0.000754





USDC to DAI Minimum Slippage


,400k,800k,1600k
200k,0.000237,0.000732,0.001813
400k,0.000000,0.000495,0.001575
800k,0.000000,0.000000,0.001073





Difference


,400k,800k,1600k
200k,-0.00004,-0.000133,-0.000458
400k,0.00000,-0.000099,-0.000420
800k,0.00000,0.000000,-0.000319


In [70]:
# This is specifically for finding min/max
def analyze_slippage():
    # Run process for both trade directions
    dfs = {}
    for x in [0, 1]:
        merged_results = []
        for amount in [200000, 400000, 800000]:
            result = []
            for amount2 in [400000, 800000, 1600000]:
                result.append((slippage[(slippage['amount'] == amount) & (slippage['dai_to_usdc'] == x)][str(amount2) + '_slippage'].min().round(5), 
                               slippage[(slippage['amount'] == amount) & (slippage['dai_to_usdc'] == x)][str(amount2) + '_slippage'].max().round(5)))
            merged_results.append(result)

        merged_results = pd.DataFrame(merged_results, columns = ['400k','800k','1600k'], index = ['200k','400k','800k'])
        if x == 0:
            dfs['USDC to DAI'] = merged_results
        else:
            dfs['DAI to USDC'] = merged_results
    return dfs

def display_analysis(dict, descriptive_string):
    print('DAI to USDC ' + descriptive_string)
    display(dict['DAI to USDC'])
    print('\n\n')
    print('USDC to DAI ' + descriptive_string)
    display(dict['USDC to DAI'])

analysis = analyze_slippage()
display_analysis(analysis, 'Min/Max Slippage')

DAI to USDC Min/Max Slippage


,400k,800k,1600k
200k,"(0.0002, 0.00035)","(0.0006, 0.00082)","(0.00135, 0.00162)"
400k,"(0.0, 0.0)","(0.0004, 0.0005)","(0.00116, 0.00134)"
800k,"(0.0, 0.0)","(0.0, 0.0)","(0.00075, 0.00084)"





USDC to DAI Min/Max Slippage


,400k,800k,1600k
200k,"(0.00024, 0.00037)","(0.00073, 0.00094)","(0.00181, 0.00207)"
400k,"(0.0, 0.0)","(0.00049, 0.00059)","(0.00158, 0.00173)"
800k,"(0.0, 0.0)","(0.0, 0.0)","(0.00107, 0.00116)"


,0,1,2
0,0.000276,0.000721,0.001515
1,0.000000,0.000445,0.001239
2,0.000000,0.000000,0.000794
